In [1]:
import numpy as np
import pandas as pd

n = 200
ordersid = np.random.randint(1, 500, size=n)
customerid = np.random.randint(583, 19573, size=n)
dates = pd.date_range('2018-11-29 12:00:00', periods=n, freq=None)
productid = np.random.randint(1, 10, size=n)
products = pd.Series([1,2,3,4,5,6,7,8,9,10])
price = pd.Series([10, 20, 100, 150, 30, 400, 5, 950, 523, 14000])
prod_pric = pd.DataFrame({'ProductId': products, 'Price':price})
pro_ord_pri = pd.DataFrame({'ProductId': productid, 'OrderId':ordersid})
order_lines = pd.merge(pro_ord_pri, prod_pric, how = 'left', on = 'ProductId')
orders = pd.DataFrame({'OrderId' : ordersid, 'CustomerId': customerid, 'DateTime': dates})

### Вопрос 1
Пожалуйста, не пишите код внутри форм ответов, разместите ссылки. Код должен открываться в браузере, без архивов, GitHub будет идеальным. Если в задании что-то непонятно, опишите возникшие вопросы и сделанные предположения. Например, в комментариях в коде.

Из базы загружены заказами клиентов магазина. Это два pandas.DataFrame’а orders и order_lines:

-orders содержит информацию о заказах: идентификатор заказа, идентификатор покупателя, время заказа:
OrderId,CustomerId,DateTime

5,583,2017-01-01 15:03:17

13,900,2019-02-05 05:02:59

69,19573,2018-11-03 23:59:59

...

-order_lines содержит информацию о линиях чеков: идентификатор продукта, идентификатор заказа, цену продукта
ProductId,OrderId,Price

5873,5,3026.0

7265,5,573.0

9675,5,159.0

5873,6,2999.0

13,6,57.0

...

Постройте отчёт (таблица в виде DataFrame), где видны
самые популярные за последний месяц продукты

суммарная выручка (за последний месяц) по каждому такому продукту

средний чек заказов (за последний месяц), в которых есть такие продукты

### Код

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
data = pd.merge(orders, order_lines, how = 'left', on = 'OrderId')

In [3]:
data = data.set_index(['DateTime'])
curr_date = datetime.today()
#не совсем понятно, что подразумевается под "за последний месяц". За 30 дней? Или за текущий неполный месяц, например за июнь?
#или же за прошлый полный месяц? решено остановиться на варианте за последние 30 дней.
last_date = curr_date - timedelta(days = 30)
data_month = data.loc[last_date:curr_date]

In [4]:
#в качестве самый популярных продуктов были определены топ-5 продуктов, которые покупались чаще всего в датафрейме
top_5_prod = pd.pivot_table(data_month, 
                       values=['OrderId'], 
                       index=['ProductId'],
                       aggfunc = lambda x: x.value_counts().count()).rename(columns = {'OrderId':'Frequency'})
top_5_prod = top_5_prod.sort_values(['Frequency'], ascending = False).iloc[0:5]

In [5]:
top_5_prod = pd.merge(top_5_prod, data_month, how = 'left', on = 'ProductId')

In [6]:
#считаем количество покупок за последний месяц по топ-продуктам
purchase_number = top_5_prod['OrderId'].value_counts().count()

In [7]:
top_5_prod = top_5_prod.drop_duplicates(subset = 'ProductId')
#выручка определяется как произведение количества товара в заказах и цены товара
top_5_prod['Revenue'] = top_5_prod['Frequency'] * top_5_prod['Price']
#средний чек заказов определяется как отношение совокупной выручки к общему числу покупок (заказов)
top_5_prod['AverageBill'] = top_5_prod['Revenue'].sum()/purchase_number
final_table = top_5_prod.reset_index().drop(['Frequency','Price', 'OrderId','index'], axis = 1)

### Финальная таблица

In [8]:
print(final_table)

   ProductId  CustomerId  Revenue  AverageBill
0          3        8904      900    98.863636
1          4       18654     1050    98.863636
2          7        8904       25    98.863636
3          2        1352       80    98.863636
4          5       12531      120    98.863636


### Тестирование

In [9]:
def test_freq():
    if top_5_prod['Frequency'].sum() <= data_month['OrderId'].sum():
        print("Frequency test is OK")
    else:
        print("Frequency test is Fail")
        
def test_purch():
    if top_5_prod['Frequency'].sum() >= purchase_number:
        print("Purchase test is OK")
    else:
        print("Purchase test is Fail")
        
def test_reven():
    if top_5_prod['Revenue'].sum() <= data_month['Price'].sum():
        print("Revenue test is OK")
    else:
        print("Revenue test is Fail")
        
def test_month():
    if data_month.reset_index()['DateTime'].value_counts().count() <= 31:
        print("Month test is OK")
    else:
        print("Month test is Fail")       

test_freq()
test_purch()
test_reven()
test_month()

Frequency test is OK
Purchase test is OK
Revenue test is OK
Month test is OK
